In [1]:
import numpy as np

In [2]:
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [3]:
import keras

In [4]:
path = keras.utils.get_file(
        '11-0.txt',
        origin='https://www.gutenberg.org/files/11/11-0.txt')

text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 164047


In [5]:
maxlen = 60
step = 3
sentences = []
next_chars = []

In [6]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

Number of sequences: 54663


In [7]:
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

Unique characters: 64


In [8]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [9]:
from keras import layers

In [10]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [11]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
import random
import sys

In [15]:
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.
            
        preds = model.predict(sampled, verbose=0) [0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        
        generated_text += next_char
        generated_text = generated_text[1:]
        
        sys.stdout.write(next_char)

epoch 1
428/428 [==============================] - 52s 122ms/step - loss: 1.8244
--- Generating with seed: "w, as we needn’t try to find any. and yet i don’t
know,” he "
------ temperature: 0.2
w, as we needn’t try to find any. and yet i don’t
know,” he ------ temperature: 0.5
w, as we needn’t try to find any. and yet i don’t
know,” he ------ temperature: 1.0
w, as we needn’t try to find any. and yet i don’t
know,” he ------ temperature: 1.2
w, as we needn’t try to find any. and yet i don’t
know,” he hawi
thayti’p juch dasting
ereso,” said the ocentever, inflegwa-lomarsto the canly.” said the gew,
arep. you ub?” therraialie.”

thenempoedve ustzy”r
thenhboiin,” apie a merarin loke,”  “thonp?” said she to to
she chsefdingissene)
“we hant,” well tulb, to she abporhhon orter.e
suchont;”

“you, and-you
knowntyw* on oo pooknonout, did opf
hinterpt of itseryout eore, or suped: that in oteprointo, andepoch 2
428/428 [==============================] - 55s 127ms/step - loss: 1.6295
--- Generatin